In [7]:
import os
from nimare import dataset, extract, io, meta

DATA_DIR = "/Users/taylor/Documents/nbc/nimare-paper/data/"

## Sleuth

In [2]:
sleuth_dset1 = io.convert_sleuth_to_dataset(
    os.path.join(DATA_DIR, "contrast-CannabisMinusControl_space-talairach_sleuth.txt")
)
sleuth_dset2 = io.convert_sleuth_to_dataset(
    os.path.join(DATA_DIR, "contrast-ControlMinusCannabis_space-talairach_sleuth.txt")
)
sleuth_dset1.save(os.path.join(DATA_DIR, "sleuth_dset1.pkl.gz"))
sleuth_dset2.save(os.path.join(DATA_DIR, "sleuth_dset2.pkl.gz"))

## Neurosynth

In [3]:
%%time
# Download the desired version of Neurosynth from GitHub.
files = extract.fetch_neurosynth(
    data_dir=DATA_DIR,
    version="7",
    source="abstract",
    vocab="terms",
    overwrite=False,
)
neurosynth_db = files[0]

# Convert the files to a Dataset.
# This may take a while (~10 minutes)
neurosynth_dset = io.convert_neurosynth_to_dataset(
    coordinates_file=neurosynth_db["coordinates"],
    metadata_file=neurosynth_db["metadata"],
    annotations_files=neurosynth_db["features"],
)
print(neurosynth_dset)

# Save the Dataset for later use.
neurosynth_dset.save(os.path.join(DATA_DIR, "neurosynth_dataset.pkl.gz"))

INFO:nimare.extract.utils:Dataset found in /Users/taylor/Documents/nbc/nimare-paper/data/neurosynth

INFO:nimare.extract.extract:Searching for any feature files matching the following criteria: [('source-abstract', 'vocab-terms', 'data-neurosynth', 'version-7')]


File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.


Dataset(14371 experiments, space='mni152_2mm')
CPU times: user 7min 32s, sys: 14.1 s, total: 7min 46s
Wall time: 7min 55s


## Neurosynth 500

In [4]:
%%time
neurosynth_dset_first_500 = neurosynth_dset.slice(neurosynth_dset.ids[:500])
print(neurosynth_dset)

neurosynth_dset_first_500.save(
    os.path.join(DATA_DIR, "neurosynth_dataset_first500.pkl.gz")
)

Dataset(14371 experiments, space='mni152_2mm')
CPU times: user 1.29 s, sys: 130 ms, total: 1.42 s
Wall time: 1.43 s


## Neurosynth 500 with MKDA MA maps

In [9]:
%%time
new_neurosynth_dset_first_500 = neurosynth_dset_first_500.copy()

kern = meta.kernel.MKDAKernel(memory_limit=None)
target_folder = os.path.join(DATA_DIR, "neurosynth_dataset_maps")

os.makedirs(target_folder, exist_ok=True)
new_neurosynth_dset_first_500.update_path(target_folder)
new_neurosynth_dset_first_500 = kern.transform(new_neurosynth_dset_first_500, return_type="dataset")
new_neurosynth_dset_first_500.save(
    os.path.join(DATA_DIR, "neurosynth_dataset_first500_with_mkda_ma.pkl.gz")
)

INFO:nimare.utils:Shared path detected: '/Users/taylor/Documents/nbc/nimare-paper/data/neurosynth_dataset_maps/'


CPU times: user 15.9 s, sys: 1.99 s, total: 17.9 s
Wall time: 18.2 s


## Add abstracts to Neurosynth 500 dataset

In [10]:
%%time
dataset_file = os.path.join(
    DATA_DIR, "neurosynth_dataset_first500_with_abstracts.pkl.gz"
)
neurosynth_dset_first_500 = extract.download_abstracts(
    neurosynth_dset_first_500,
    email="tsalo006@fiu.edu",
)
neurosynth_dset_first_500.save(dataset_file)

INFO:nimare.extract.extract:Downloading chunk 1 of 1


CPU times: user 977 ms, sys: 46.4 ms, total: 1.02 s
Wall time: 6.34 s


## NeuroQuery

In [11]:
%%time
# Download the desired version of NeuroQuery from GitHub.
files = extract.fetch_neuroquery(
    data_dir=DATA_DIR,
    version="1",
    source="combined",
    vocab="neuroquery6308",
    type="tfidf",
    overwrite=False,
)
neuroquery_db = files[0]

# Convert the files to a Dataset.
# This may take a while (~10 minutes)
neuroquery_dset = io.convert_neurosynth_to_dataset(
    coordinates_file=neuroquery_db["coordinates"],
    metadata_file=neuroquery_db["metadata"],
    annotations_files=neuroquery_db["features"],
)
print(neuroquery_dset)

# Save the Dataset for later use.
neuroquery_dset.save(os.path.join(DATA_DIR, "neuroquery_dataset.pkl.gz"))

INFO:nimare.extract.utils:Dataset found in /Users/taylor/Documents/nbc/nimare-paper/data/neuroquery

INFO:nimare.extract.extract:Searching for any feature files matching the following criteria: [('source-combined', 'vocab-neuroquery6308', 'type-tfidf', 'data-neuroquery', 'version-1')]


File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.


Dataset(13459 experiments, space='mni152_2mm')
CPU times: user 8min 57s, sys: 34.3 s, total: 9min 31s
Wall time: 9min 52s


## Pain dataset of IBMAs

In [12]:
%%time
from nimare.tests.utils import get_test_data_path

img_dset_file = os.path.join(DATA_DIR, "nidm_dset.pkl.gz")
dset_dir = extract.download_nidm_pain(
    data_dir=DATA_DIR,
    overwrite=False,
)
dset_file = os.path.join(get_test_data_path(), "nidm_pain_dset.json")
img_dset = dataset.Dataset(dset_file)

# Point the Dataset toward the images we've downloaded
img_dset.update_path(dset_dir)
img_dset.save(img_dset_file)

INFO:nimare.extract.utils:Dataset found in /Users/taylor/Documents/nbc/nimare-paper/data/nidm_21pain

INFO:nimare.utils:Shared path detected: '/Users/taylor/Documents/nbc/nimare-paper/data/nidm_21pain/'
